In [154]:
import win32com.client
import pandas as pd
from tqdm import tqdm
import time
import json
import numpy as np
import ast

In [155]:
PATH = 'c:\\Users\\passi\\Desktop\\programming\\stair_to_heaven\\'
df = pd.read_csv(PATH + "catch_highest/data/extracted_data/kosdaq_150.csv", usecols = ['date','open','close'])

In [156]:
#분봉 데이터 파일들을 모두 리스트에 넣기
file_list = os.listdir(PATH + "catch_highest/data/minute_stock_data/")
file_list.sort()
if '.DS_Store' in file_list:
    file_list = file_list[1:]
else:
    None

#처음날짜
for i in range(len(file_list)):
    if file_list[i].startswith('20190513'): #20190513
        file_list = file_list[i:]
        break
#끝날짜
for i in reversed(range(len(file_list))):
    if file_list[i].startswith('20201030'):
        file_list = file_list[:i+1]
        break


In [157]:
# 우선주만 보자
new_file_list = []
for i in range(len(file_list)):
    if file_list[i].endswith('우.csv'):
        new_file_list.append(file_list[i])

file_list = new_file_list

In [158]:
# #하락장일때만 보자
# df = pd.read_csv(PATH + "catch_highest/data/extracted_data/kosdaq_150.csv", usecols = ['date','open','close'])
# array = []
# for i in range(df.shape[0]-1):
#     if df['open'][i] < df['close'][i+1]:
#         array.append(df['date'][i])
# array = list(map(str, array))

# new_file_list = []
# for i in range(len(file_list)):
#     if file_list[i][0:8] in array:
#         new_file_list.append(file_list[i])

# file_list = new_file_list

In [159]:
start_time = 901
df = pd.read_csv(PATH + "catch_highest/data/extracted_data/only_time.csv", usecols = ['time'])
for i in range(df.shape[0]):
    if df['time'][i] == start_time:
        df = pd.DataFrame(df['time'][i:]).reset_index(drop=True)
        break

for file in tqdm(file_list):
    file = file[0:-4]
    test = pd.read_csv(PATH + "catch_highest/data/minute_stock_data/%s.csv" % file, usecols=['time','open'])
    test.columns = ['time',file]
    test[file] = test[file].astype('int')
    # 단일가 거래 거르고
    if test.shape[0] < 50:
        continue
    # 동전주 거르고
    if len(str(test[file][0])) == 3:
        continue
        
    df = pd.merge(df,test, on='time', how = 'left')
    # 중간중간 vi 걸린 것들은 다음값으로 채워주고
    for i in range(df.shape[0]):
        if np.isnan(df[file][i]):
            index = i 
            stopping = True
            try:
                while stopping:
                    if np.isnan(df[file][index+1]):
                        index += 1
                    else:
                        valid = df[file][index+1]
                        stopping = False
                df[file][i] = valid
            except:
                continue
    for j in range(1,df.shape[0]):
        value = round((df[file][j] - df[file][0]) / df[file][0],4) - 0.0025
        # if value <= -0.05:
        #     value = -0.05
        #     df[file][j:] = value
        # else:
        df[file][j] = value

#해당 날짜에 값이 없는 경우에는 모두 0으로 대체
df = df.fillna(0)

#날짜와 기업 자르고 다시 재정렬
df['date'] = [str() for x in range(len(df.index))]
df['company'] = [str() for x in range(len(df.index))]

df.to_csv(PATH + "catch_highest/data/final_result/test_3.csv")

for i in range(df.shape[0]):
    df['date'][i] = df['time'][i][0:8]
    df['company'][i] = df['time'][i][9:]

cols = df.columns.tolist()
cols = [cols[-2]] + cols[1:-2]
df = df.reindex(columns=cols)


#날짜별 기업들의 수익률 평균내기
df = df.groupby('date').mean().reset_index()
df.to_csv(PATH + "catch_highest/data/final_result/test_1.csv")


# date와 나머지 열을 쪼개서 각각 temp, temp2에 저장
temp = df.iloc[:,0]
temp2 = df.iloc[:,1:]

#숫자만 들어가 있는 temp2에서 복리 계산을 진행
for j in tqdm(temp2.columns):
    temp2[j][0] = 100*(1+temp2[j][0])
    for i in range(1, temp2.shape[0]):
        temp2[j][i] = temp2[j][i-1] * (1+temp2[j][i])

#이제 다시 쪼개진 데이터프레임들 결합
df = pd.concat([temp,temp2],axis=1)
df.to_csv(PATH + "catch_highest/data/final_result/test_2.csv")

#수익이 가장 극대화되는 시간대와 수익률
df = pd.DataFrame(df.iloc[-1])
df = df.T
target_time = int(df.iloc[:,1:].max(axis=0).idxmax())
profit = round((df.iloc[:,1:].max(axis=0).max()/100),2)
print('가장 수익률이 극대화되는 시간은 ' + str(target_time) + '이며, 수익률은 대략 ' + str(profit) + '배 정도 됩니다.')

100%|██████████| 295/295 [00:16<00:00, 18.41it/s]


IndexError: invalid index to scalar variable.